In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

torch.manual_seed(10)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# ============================ step 1/5 生成数据 ============================
sample_nums = 100
mean_value = 1.7
bias = 1
n_data = torch.ones(size=(sample_nums, 2))
x0 = torch.normal(mean=mean_value * n_data, std=1) + bias
y0 = torch.zeros(size=(sample_nums,))
x1 = torch.normal(mean=-mean_value * n_data, std=1) + bias
y1 = torch.ones(size=(sample_nums,))

train_x = torch.cat(tensors=[x0, x1], dim=0)
train_y = torch.cat(tensors=[y0, y1], dim=0)
train_x = train_x.to(device)
print("train_x:{}".format(train_x))
train_y = train_y.to(device)
print("train_y:{}".format(train_y))

In [ ]:
 # ============================ step 2/5 选择模型 ============================
class LR(nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.features = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.features(x)
        x = self.sigmoid(x)
        return x
    
lr_net = LR()
lr_net = lr_net.to(device=device)

In [ ]:
# ============================ step 3/5 选择损失函数 ============================
loss_fn = nn.BCELoss()
# ============================ step 4/5 选择优化器   ============================
lr = 0.01
optimizer = torch.optim.SGD(params=lr_net.parameters(), lr=lr, momentum=0.9)

In [ ]:
# ============================ step 5/5 模型训练 ============================
# plt.ion()
for iteration in range(1000):
    # 前向传播
    y_pred = lr_net.forward(x=train_x)
    # print("y_pred:{}".format(y_pred))
    # 计算 loss
    # print("y_pred.squeeze() shape is:{}".format(y_pred.squeeze().shape))
    # print("train_y shape is:{}".format(train_y.shape))
    loss = loss_fn(y_pred.squeeze(), train_y)
    # 反向传播
    loss.backward()
    # 更新参数
    optimizer.step()
    # 清空梯度
    optimizer.zero_grad()
    # 绘图
    if iteration % 20 == 0:
        mask = y_pred.ge(0.5).float().squeeze()  # 以0.5为阈值进行分类
        correct = (mask == train_y).sum()  # 计算正确预测的样本个数
        acc = correct.item() / train_y.size(0)  # 计算分类准确率

        plt.scatter(x0.data.numpy()[:, 0], x0.data.numpy()[:, 1], c='r', label='class 0')
        plt.scatter(x1.data.numpy()[:, 0], x1.data.numpy()[:, 1], c='b', label='class 1')

        w0, w1 = lr_net.features.weight[0]
        w0, w1 = float(w0.item()), float(w1.item())
        plot_b = float(lr_net.features.bias[0].item())
        plot_x = np.arange(-6, 6, 0.1)
        # ?
        plot_y = (-w0 * plot_x - plot_b) / w1

        plt.xlim(-5, 7)
        plt.ylim(-7, 7)
        plt.plot(plot_x, plot_y)

        plt.text(x=-5, y=5, s='Loss=%.4f' % loss.data.item(), fontdict={'size': 20, 'color': 'red'})
        plt.title("Iteration: {}\nw0:{:.2f} w1:{:.2f} b: {:.2f} accuracy:{:.2%}".format(iteration, w0, w1, plot_b, acc))
        plt.legend()
        # plt.savefig(str(iteration / 20)+".png")
        plt.show()
        plt.pause(0.5)
        # plt.clf()
        # 如果准确率大于 99%，则停止训练
        if acc > 0.99:
            break